## Dependencies

In [80]:
!pip install opencv-python mediapipe sklearn matplotlib tensorflow==2.12.0

In [81]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## Getting Hand Keypoints

In [51]:
mp_hands = mp.solutions.hands #hands model
mp_drawing = mp.solutions.drawing_utils

In [52]:
def mediapie_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    return image, results

In [62]:
def draw_landmarks(image, results):
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                mp_drawing.draw_landmarks(image, handLms, mp_hands.HAND_CONNECTIONS)


In [172]:
cap = cv2.VideoCapture(0)
#mediapipe model
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.1) as hands:
    while cap.isOpened():

        ret, frame = cap.read()
        
        #Run detection
        image, results = mediapie_detection(frame, hands)
        
        #Draw Detections
        draw_landmarks(image, results)

        cv2.imshow("Frame", image)
        
        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

In [61]:
cap.release()
cv2.destroyAllWindows()

In [173]:
print(results.multi_hand_landmarks[0])

TypeError: 'NoneType' object is not subscriptable

## Getting Keypoint Values

In [120]:

len(results.multi_hand_landmarks[0].landmark)
    

21

In [151]:
all_landmarks = []
for n in range(len(results.multi_hand_landmarks[0].landmark)):
    test = np.array([results.multi_hand_landmarks[0].landmark[n].x,
                     results.multi_hand_landmarks[0].landmark[n].y,
                     results.multi_hand_landmarks[0].landmark[n].z])
    all_landmarks.append(test)

In [174]:
all_hand_landmarks = np.array([[results.multi_hand_landmarks[0].landmark[n].x, 
                  results.multi_hand_landmarks[0].landmark[n].y,
                  results.multi_hand_landmarks[0].landmark[n].z] 
                  for n in range(len(results.multi_hand_landmarks[0].landmark))]).flatten() if results.multi_hand_landmarks else np.zeros(21 * 3)

In [175]:
all_hand_landmarks

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [178]:
def extract_keypoints(results):
    all_hand_landmarks = np.array([[results.multi_hand_landmarks[0].landmark[n].x, 
                  results.multi_hand_landmarks[0].landmark[n].y,
                  results.multi_hand_landmarks[0].landmark[n].z] 
                  for n in range(len(results.multi_hand_landmarks[0].landmark))]).flatten() if results.multi_hand_landmarks else np.zeros(21 * 3)
    return all_hand_landmarks

In [179]:
extract_keypoints(results).shape

(63,)

## Setup Folders for Collection

In [182]:
DATA_PATH = os.path.join('MP_data')
actions = np.array(['right', 'left', 'up', 'down', 'hit']) #actions

no_sequences = 30 # number of videos worth of data

sequence_length = 30 # number of frames in each sequence

In [184]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## Collect Keypoint Values for Training


In [185]:
cap = cv2.VideoCapture(0)
# Mediapipe model
with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.1) as hands:
    
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):       
        
                ret, frame = cap.read()

                #Run detection
                image, results = mediapie_detection(frame, hands)

                #Draw Detections
                draw_landmarks(image, results)
                
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collection frames for {} Video Number {}'.format(action, sequence), (15, 12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                    
                    cv2.waitKey(2000)
                else:
                     cv2.putText(image, 'Collection frames for {} Video Number {}'.format(action, sequence), (15, 12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                       
                #Saving Data
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                        
                cv2.imshow("Frame", image)

                if cv2.waitKey(10) & 0xff == ord('q'):
                    break
    
cap.release()
cv2.destroyAllWindows()

In [186]:
results_test = extract_keypoints(results)

In [187]:
results_test

array([ 5.76459885e-01,  8.62514138e-01,  7.76904812e-07,  6.09725356e-01,
        7.28623807e-01,  1.44570880e-02,  6.79190040e-01,  6.57678723e-01,
        9.31228790e-03,  7.53093421e-01,  6.72762215e-01,  1.84059842e-03,
        7.97541380e-01,  7.24783182e-01, -7.35857617e-03,  7.37519860e-01,
        6.35675251e-01, -3.76042649e-02,  8.55379760e-01,  6.24321759e-01,
       -4.59168106e-02,  9.19230580e-01,  6.22275949e-01, -4.62355576e-02,
        9.68281388e-01,  6.21582687e-01, -4.44501713e-02,  7.65916824e-01,
        7.16073096e-01, -4.95637059e-02,  8.87605131e-01,  6.98869526e-01,
       -4.74913232e-02,  9.57682848e-01,  6.89162433e-01, -4.56021987e-02,
        1.00629735e+00,  6.82675779e-01, -4.54669595e-02,  7.79211223e-01,
        8.08820367e-01, -5.43670282e-02,  8.17399919e-01,  8.05412591e-01,
       -3.96791920e-02,  7.87849486e-01,  8.08348179e-01, -2.44618170e-02,
        7.53121376e-01,  8.14903319e-01, -1.51992403e-02,  7.78916538e-01,
        8.92888606e-01, -

In [188]:
np.save('0', results_test)

In [189]:
np.load('0.npy')

array([ 5.76459885e-01,  8.62514138e-01,  7.76904812e-07,  6.09725356e-01,
        7.28623807e-01,  1.44570880e-02,  6.79190040e-01,  6.57678723e-01,
        9.31228790e-03,  7.53093421e-01,  6.72762215e-01,  1.84059842e-03,
        7.97541380e-01,  7.24783182e-01, -7.35857617e-03,  7.37519860e-01,
        6.35675251e-01, -3.76042649e-02,  8.55379760e-01,  6.24321759e-01,
       -4.59168106e-02,  9.19230580e-01,  6.22275949e-01, -4.62355576e-02,
        9.68281388e-01,  6.21582687e-01, -4.44501713e-02,  7.65916824e-01,
        7.16073096e-01, -4.95637059e-02,  8.87605131e-01,  6.98869526e-01,
       -4.74913232e-02,  9.57682848e-01,  6.89162433e-01, -4.56021987e-02,
        1.00629735e+00,  6.82675779e-01, -4.54669595e-02,  7.79211223e-01,
        8.08820367e-01, -5.43670282e-02,  8.17399919e-01,  8.05412591e-01,
       -3.96791920e-02,  7.87849486e-01,  8.08348179e-01, -2.44618170e-02,
        7.53121376e-01,  8.14903319e-01, -1.51992403e-02,  7.78916538e-01,
        8.92888606e-01, -

## Preprocess Data and Create Labelmap and Features

In [190]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [191]:
label_map = {label:num for num, label in enumerate(actions)}

In [192]:
label_map

{'right': 0, 'left': 1, 'up': 2, 'down': 3, 'hit': 4}

In [200]:
sequences, labels = [], []

for action in actions:
        for sequence in range(no_sequences):
            window = []
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])

In [203]:
np.array(sequences).shape

(150, 30, 63)

In [204]:
np.array(labels).shape

(150,)

In [205]:
X = np.array(sequences)

In [208]:
X.shape

(150, 30, 63)

In [206]:
y = to_categorical(labels).astype(int)

In [207]:
y

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0,

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [211]:
X_train.shape

(142, 30, 63)

## Building and Training LSTM Neural Network

In [212]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [215]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [222]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [223]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=['categorical_accuracy'])

In [224]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
5/5 [==============================] - 2s 25ms/step - loss: 1.6036 - categorical_accuracy: 0.2183
Epoch 2/2000
5/5 [==============================] - 0s 24ms/step - loss: 1.5420 - categorical_accuracy: 0.3099
Epoch 3/2000
5/5 [==============================] - 0s 24ms/step - loss: 1.5017 - categorical_accuracy: 0.4366
Epoch 4/2000
5/5 [==============================] - 0s 24ms/step - loss: 1.4181 - categorical_accuracy: 0.4437
Epoch 5/2000
5/5 [==============================] - 0s 25ms/step - loss: 1.2188 - categorical_accuracy: 0.4296
Epoch 6/2000
5/5 [==============================] - 0s 25ms/step - loss: 1.5269 - categorical_accuracy: 0.4859
Epoch 7/2000
5/5 [==============================] - 0s 24ms/step - loss: 1.5404 - categorical_accuracy: 0.4085
Epoch 8/2000
5/5 [==============================] - 0s 24ms/step - loss: 1.3514 - categorical_accuracy: 0.4366
Epoch 9/2000
5/5 [==============================] - 0s 25ms/step - loss: 1.2875 - categorical_accuracy: 0.4577
E

5/5 [==============================] - 0s 25ms/step - loss: 0.4693 - categorical_accuracy: 0.8239
Epoch 75/2000
5/5 [==============================] - 0s 26ms/step - loss: 0.4513 - categorical_accuracy: 0.8451
Epoch 76/2000
5/5 [==============================] - 0s 26ms/step - loss: 0.7097 - categorical_accuracy: 0.6972
Epoch 77/2000
5/5 [==============================] - 0s 25ms/step - loss: 0.6734 - categorical_accuracy: 0.6972
Epoch 78/2000
5/5 [==============================] - 0s 25ms/step - loss: 1.1296 - categorical_accuracy: 0.5915
Epoch 79/2000
5/5 [==============================] - 0s 26ms/step - loss: 0.9450 - categorical_accuracy: 0.5775
Epoch 80/2000
5/5 [==============================] - 0s 26ms/step - loss: 0.7626 - categorical_accuracy: 0.7394
Epoch 81/2000
5/5 [==============================] - 0s 26ms/step - loss: 0.8010 - categorical_accuracy: 0.6690
Epoch 82/2000
5/5 [==============================] - 0s 25ms/step - loss: 0.7063 - categorical_accuracy: 0.7817
Epoch 

5/5 [==============================] - 0s 25ms/step - loss: 0.9072 - categorical_accuracy: 0.6690
Epoch 147/2000
5/5 [==============================] - 0s 28ms/step - loss: 0.8659 - categorical_accuracy: 0.7042
Epoch 148/2000
5/5 [==============================] - 0s 26ms/step - loss: 0.8296 - categorical_accuracy: 0.6901
Epoch 149/2000
5/5 [==============================] - 0s 26ms/step - loss: 0.7991 - categorical_accuracy: 0.6972
Epoch 150/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.7522 - categorical_accuracy: 0.7183
Epoch 151/2000
5/5 [==============================] - 0s 28ms/step - loss: 0.6956 - categorical_accuracy: 0.7465
Epoch 152/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.6390 - categorical_accuracy: 0.7535
Epoch 153/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.5858 - categorical_accuracy: 0.7676
Epoch 154/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.5537 - categorical_accuracy: 0.823

5/5 [==============================] - 0s 28ms/step - loss: 0.1455 - categorical_accuracy: 0.9718
Epoch 219/2000
5/5 [==============================] - 0s 28ms/step - loss: 0.1287 - categorical_accuracy: 0.9718
Epoch 220/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.1246 - categorical_accuracy: 0.9648
Epoch 221/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.1206 - categorical_accuracy: 0.9648
Epoch 222/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.1702 - categorical_accuracy: 0.9366
Epoch 223/2000
5/5 [==============================] - 0s 26ms/step - loss: 0.2162 - categorical_accuracy: 0.9437
Epoch 224/2000
5/5 [==============================] - 0s 27ms/step - loss: 0.1915 - categorical_accuracy: 0.9437
Epoch 225/2000
5/5 [==============================] - 0s 29ms/step - loss: 0.1848 - categorical_accuracy: 0.9437
Epoch 226/2000
5/5 [==============================] - 0s 29ms/step - loss: 0.1619 - categorical_accuracy: 0.964

5/5 [==============================] - 0s 33ms/step - loss: 0.0907 - categorical_accuracy: 0.9859
Epoch 291/2000
5/5 [==============================] - 0s 31ms/step - loss: 0.0983 - categorical_accuracy: 0.9648
Epoch 292/2000
5/5 [==============================] - 0s 31ms/step - loss: 0.1491 - categorical_accuracy: 0.9437
Epoch 293/2000
5/5 [==============================] - 0s 30ms/step - loss: 0.1307 - categorical_accuracy: 0.9507
Epoch 294/2000
5/5 [==============================] - 0s 31ms/step - loss: 0.1315 - categorical_accuracy: 0.9577
Epoch 295/2000
5/5 [==============================] - 0s 30ms/step - loss: 0.3360 - categorical_accuracy: 0.9014
Epoch 296/2000
5/5 [==============================] - 0s 31ms/step - loss: 0.5112 - categorical_accuracy: 0.8592
Epoch 297/2000
5/5 [==============================] - 0s 29ms/step - loss: 0.5088 - categorical_accuracy: 0.8028
Epoch 298/2000
5/5 [==============================] - 0s 28ms/step - loss: 0.2714 - categorical_accuracy: 0.922

...

5/5 [==============================] - 0s 28ms/step - loss: 3.2489e-07 - categorical_accuracy: 1.0000
Epoch 1687/2000
5/5 [==============================] - 0s 29ms/step - loss: 3.2321e-07 - categorical_accuracy: 1.0000
Epoch 1688/2000
5/5 [==============================] - 0s 28ms/step - loss: 3.2153e-07 - categorical_accuracy: 1.0000
Epoch 1689/2000
5/5 [==============================] - 0s 28ms/step - loss: 3.1985e-07 - categorical_accuracy: 1.0000
Epoch 1690/2000
5/5 [==============================] - 0s 28ms/step - loss: 3.1985e-07 - categorical_accuracy: 1.0000
Epoch 1691/2000
5/5 [==============================] - 0s 28ms/step - loss: 3.1985e-07 - categorical_accuracy: 1.0000
Epoch 1692/2000
5/5 [==============================] - 0s 28ms/step - loss: 3.1817e-07 - categorical_accuracy: 1.0000
Epoch 1693/2000
5/5 [==============================] - 0s 28ms/step - loss: 3.1649e-07 - categorical_accuracy: 1.0000
Epoch 1694/2000
5/5 [==============================] - 0s 28ms/step - lo

5/5 [==============================] - 0s 27ms/step - loss: 2.6108e-07 - categorical_accuracy: 1.0000
Epoch 1756/2000
5/5 [==============================] - 0s 27ms/step - loss: 2.5773e-07 - categorical_accuracy: 1.0000
Epoch 1757/2000
5/5 [==============================] - 0s 27ms/step - loss: 2.5773e-07 - categorical_accuracy: 1.0000
Epoch 1758/2000
5/5 [==============================] - 0s 28ms/step - loss: 2.5773e-07 - categorical_accuracy: 1.0000
Epoch 1759/2000
5/5 [==============================] - 0s 27ms/step - loss: 2.5605e-07 - categorical_accuracy: 1.0000
Epoch 1760/2000
5/5 [==============================] - 0s 28ms/step - loss: 2.5437e-07 - categorical_accuracy: 1.0000
Epoch 1761/2000
5/5 [==============================] - 0s 28ms/step - loss: 2.5269e-07 - categorical_accuracy: 1.0000
Epoch 1762/2000
5/5 [==============================] - 0s 28ms/step - loss: 2.5185e-07 - categorical_accuracy: 1.0000
Epoch 1763/2000
5/5 [==============================] - 0s 27ms/step - lo

5/5 [==============================] - 0s 29ms/step - loss: 2.0484e-07 - categorical_accuracy: 1.0000
Epoch 1825/2000
5/5 [==============================] - 0s 29ms/step - loss: 2.0484e-07 - categorical_accuracy: 1.0000
Epoch 1826/2000
5/5 [==============================] - 0s 28ms/step - loss: 2.0400e-07 - categorical_accuracy: 1.0000
Epoch 1827/2000
5/5 [==============================] - 0s 29ms/step - loss: 2.0316e-07 - categorical_accuracy: 1.0000
Epoch 1828/2000
5/5 [==============================] - 0s 28ms/step - loss: 2.0316e-07 - categorical_accuracy: 1.0000
Epoch 1829/2000
5/5 [==============================] - 0s 28ms/step - loss: 2.0232e-07 - categorical_accuracy: 1.0000
Epoch 1830/2000
5/5 [==============================] - 0s 28ms/step - loss: 2.0232e-07 - categorical_accuracy: 1.0000
Epoch 1831/2000
5/5 [==============================] - 0s 29ms/step - loss: 2.0148e-07 - categorical_accuracy: 1.0000
Epoch 1832/2000
5/5 [==============================] - 0s 28ms/step - lo

5/5 [==============================] - 0s 28ms/step - loss: 1.6454e-07 - categorical_accuracy: 1.0000
Epoch 1894/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.6454e-07 - categorical_accuracy: 1.0000
Epoch 1895/2000
5/5 [==============================] - 0s 28ms/step - loss: 1.6454e-07 - categorical_accuracy: 1.0000
Epoch 1896/2000
5/5 [==============================] - 0s 28ms/step - loss: 1.6286e-07 - categorical_accuracy: 1.0000
Epoch 1897/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.6118e-07 - categorical_accuracy: 1.0000
Epoch 1898/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.6118e-07 - categorical_accuracy: 1.0000
Epoch 1899/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.6118e-07 - categorical_accuracy: 1.0000
Epoch 1900/2000
5/5 [==============================] - 0s 28ms/step - loss: 1.6118e-07 - categorical_accuracy: 1.0000
Epoch 1901/2000
5/5 [==============================] - 0s 28ms/step - lo

5/5 [==============================] - 0s 29ms/step - loss: 1.3180e-07 - categorical_accuracy: 1.0000
Epoch 1963/2000
5/5 [==============================] - 0s 28ms/step - loss: 1.3096e-07 - categorical_accuracy: 1.0000
Epoch 1964/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.3180e-07 - categorical_accuracy: 1.0000
Epoch 1965/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.3012e-07 - categorical_accuracy: 1.0000
Epoch 1966/2000
5/5 [==============================] - 0s 28ms/step - loss: 1.3012e-07 - categorical_accuracy: 1.0000
Epoch 1967/2000
5/5 [==============================] - 0s 28ms/step - loss: 1.3096e-07 - categorical_accuracy: 1.0000
Epoch 1968/2000
5/5 [==============================] - 0s 28ms/step - loss: 1.3012e-07 - categorical_accuracy: 1.0000
Epoch 1969/2000
5/5 [==============================] - 0s 27ms/step - loss: 1.3012e-07 - categorical_accuracy: 1.0000
Epoch 1970/2000
5/5 [==============================] - 0s 28ms/step - lo

In [225]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 30, 64)            32768     
                                                                 
 lstm_8 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_9 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 5)                 165       
                                                                 
Total params: 187,397
Trainable params: 187,397
Non-tr

In [227]:
res = model.predict(X_test)

1/1 [==============================] - 0s 17ms/step


In [228]:
np.sum(res[0])

1.0

In [250]:
actions[np.argmax(res[1])]

'right'

In [249]:
actions[np.argmax(y_test[1])]

'right'

In [253]:
model.save('action.h5')

## Evaluating Accuracy

In [254]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [255]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 19ms/step


In [256]:
yTrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [257]:
multilabel_confusion_matrix(yTrue, yhat)

array([[[7, 0],
        [0, 1]],

       [[7, 0],
        [1, 0]],

       [[7, 1],
        [0, 0]],

       [[6, 0],
        [0, 2]],

       [[4, 0],
        [0, 4]]], dtype=int64)

## Testing my Model

In [303]:
colors = [(144, 238, 144), (230, 216, 173), (224, 225, 225), (227, 195, 203), (0, 165, 255)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
    
        print(prob)
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    return output_frame

In [304]:
threshold = 0.4
sequence = []

cap = cv2.VideoCapture(0)
#mediapipe model
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.1) as hands:
    while cap.isOpened():

        ret, frame = cap.read()
        
        #Run detection
        image, results = mediapie_detection(frame, hands)
        
        #Draw Detections
        draw_landmarks(image, results)
        
        #Prediction
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            print(res.max)
            
        image = prob_viz(res, actions, image, colors)
            

        cv2.imshow("Frame", image)
        
        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761

1/1 [==============================] - 0s 15ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A40BDB8270>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 17ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A40BDB8E10>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 15ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A40BDB85D0>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 15ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A40C54F1B0>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 16ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A40BDB85D0>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 16ms/step
rig

1/1 [==============================] - 0s 15ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A41CD19510>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 15ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A40E310330>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 15ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A40C54F570>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 17ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A41573EED0>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 16ms/step
right
<built-in method max of numpy.ndarray object at 0x000001A40BDD6E70>
0.9999993
5.586377e-14
8.664995e-08
1.3972322e-18
5.761349e-07
1/1 [==============================] - 0s 16ms/step
rig

In [267]:
cap.release()
cv2.destroyAllWindows()

In [279]:
res.max??

In [280]:
print(actions)

['right' 'left' 'up' 'down' 'hit']


In [281]:
print(res)

[9.9999928e-01 5.5863771e-14 8.6649948e-08 1.3972322e-18 5.7613488e-07]


In [284]:
len(res)

5